In [3]:
import os
import random
from time import time

import numpy as np
from IPython.display import Markdown, display
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

try:
    import tensorflow_similarity as tfsim
except ModuleNotFoundError:
    !pip install tensorflow_similarity
    import tensorflow_similarity as tfsim

tfsim.utils.tf_cap_memory()  # Avoid GPU memory blow up

NameError: name 'tfsim' is not defined

In [4]:
IMG_SIZE = 300  # slightly larger than EfficienNetB0 size to allow random crops.
TARGET_IMG_SIZE = 224

def resize(img, label, shape=(IMG_SIZE, IMG_SIZE)):
    """Resize images to required shape."""
    with tf.device("/cpu:0"):
        img = tf.cast(img, dtype="int32")
        img = tf.image.resize_with_pad(img, *shape)
        return img, label

def img_augmentation(img_batch, y, *args):
    """Randomize image shape and orientation"""
    img_batch = tf.keras.layers.RandomCrop(TARGET_IMG_SIZE, TARGET_IMG_SIZE)(img_batch)
    img_batch = tf.image.random_flip_left_right(img_batch)
    return img_batch, y

**Dataset**

In [14]:
MIN_NO_CLASSES = 20  #@param {type:"integer"}
EXAMPLES_PER_CLASS_PER_BATCH = 4  #@param {type:"slider", min:1, max:20, step:1}

num_of_classes = tfds.image_classification.Cifar100()._info().features['label'].num_classes
ds_name = tfds.image_classification.Cifar100.name

cifar100


In [ ]:
training_classes = int(num_of_classes * 0.6)
train_cls = random.sample(range(num_of_classes), k=training_classes)
test_cls = [cls_id for cls_id in range(num_of_classes) if cls_id not in train_cls]

print(f"Class IDs seen during training {train_cls}")

# use the train split for training
train_ds = tfsim.samplers.TFDatasetMultiShotMemorySampler(
    ds_name,
    splits="train",
    examples_per_class_per_batch=EXAMPLES_PER_CLASS_PER_BATCH,
    classes_per_batch=training_classes,
    preprocess_fn=resize,
    class_list=train_cls,
    augmenter=img_augmentation,
)

# use the test split for indexing and querying
test_ds = tfsim.samplers.TFDatasetMultiShotMemorySampler(
    ds_name,
    splits="test",
    total_examples_per_class=20,
    classes_per_batch=training_classes,
    class_list=test_cls,
    preprocess_fn=resize,
)

In [ ]:
NUM_TARGETS = 200  #@param {type:"integer"}
NUM_QUERIES = 300  #@param {type:"integer"}
k = 3  #@param {type:"integer"}
log_dir = "logs/%d/" % (time())

In [ ]:
# Setup EvalCallback by splitting the test data into targets and queries.
queries_x, queries_y = test_ds.get_slice(0, NUM_QUERIES)
targets_x, targets_y = test_ds.get_slice(NUM_QUERIES, NUM_TARGETS)
tsc = tfsim.callbacks.EvalCallback(
    queries_x,
    queries_y,
    targets_x,
    targets_y,
    metrics=["f1"],
    k=k,
    # tb_logdir=log_dir  # uncomment if you want to track in tensorboard
)

# Setup an EvalCallback for a known and unknown class split.
val_loss = tfsim.callbacks.EvalCallback(
    queries_x,
    queries_y,
    targets_x,
    targets_y,
    metrics=["binary_accuracy"],
    known_classes=tf.constant(train_cls),
    k=k,
    # tb_logdir=log_dir  # uncomment if you want to track in tensorboard
)

# Adding the Tensorboard callback to track metrics in tensorboard.
# tbc = tf.keras.callbacks.TensorBoard(log_dir=log_dir) # uncomment if you want to track in tensorboard

callbacks = [
    val_loss,
    tsc,
    # tbc # uncomment if you want to track in tensorboard
]

**Model**
Hyper-parameters

In [ ]:
EMBEDDING_SIZE = 128  #@param {type:"slider", min:32, max:1024, step:32}
epochs = 5  #@param {type:"slider", min:1, max:64, step:1}
LR = 0.0001
steps_per_epoch = 100  #@param {type:"integer"}
val_steps = 50  #@param {type:"integer"}
positive_mining_strategy='hard'  #@param ['hard', 'easy']
negative_mining_strategy='semi-hard'  #@param ['hard', 'semi-hard', 'easy']

In [ ]:
model = tfsim.architectures.EfficientNetSim(
    train_ds.example_shape,
    EMBEDDING_SIZE,
    pooling="gem",    # Can change to use `gem` -> GeneralizedMeanPooling2D
    gem_p=3.0,        # Increase the contrast between activations in the feature map.
)

**Training**

In [ ]:
loss = tfsim.losses.TripletLoss(distance='cosine',
                                positive_mining_strategy=positive_mining_strategy,
                                negative_mining_strategy=negative_mining_strategy)

model.compile(optimizer=tf.keras.optimizers.Adam(LR), loss=loss)
history = model.fit(
    train_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_ds,
    validation_steps=val_steps,
    callbacks=callbacks,
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["loss", "val_loss"])
plt.title(f"Loss: {loss.name} - LR: {LR}")
plt.show()

In [ ]:
plt.plot(history.history["binary_accuracy_known_classes"])
plt.plot(history.history["binary_accuracy_unknown_classes"])
plt.legend(["binary_accuracy_known", "binary_accuracy_unknown"])
plt.title(f"Known | Unknown binary_accuracy: {loss.name} - LR: {LR}")
plt.show()

In [ ]:
# What is indexed
index_size = 360
query_size = 360
index_x, index_y = test_ds.get_slice(0, index_size)
index_data = tf.cast(index_x, dtype="int32")

# what will be used as never seen before queries to test performance
test_x, test_y = test_ds.get_slice(index_size, query_size)
test_y = [int(c) for c in test_y]
test_data = tf.cast(test_x, dtype="int32")

In [ ]:
model.reset_index()
model.index(index_x, index_y, data=index_data)

In [ ]:
num_examples = 5  #@param {type:"slider", min:1, max:20, step:1}
num_neigboors = 5  #@param {type:"slider", min:1, max:20, step:1}
idxs = random.sample(range(len(test_y)), num_examples)
batch = tf.gather(test_x, idxs)
nns = model.lookup(batch, k=num_neigboors)
for bid, nn in zip(idxs, nns):
    # view results close by
    if test_y[bid] in train_cls:
        display(Markdown("**Known Class**"))
    else:
        display(Markdown("**Unknown Class**"))
    tfsim.visualization.viz_neigbors_imgs(test_data[bid], test_y[bid], nn, cmap='Grays')

In [ ]:
num_examples_to_clusters = 720  # @param {type:"integer"}
thumb_size = 96  # @param {type:"integer"}
plot_size = 800
vx, vy = test_ds.get_slice(0, num_examples_to_clusters)
tfsim.visualization.projector(
    model.predict(vx), labels=vy, images=vx, image_size=thumb_size, plot_size=plot_size
)

In [ ]:
num_calibration_samples = 1000  # @param {type:"integer"}
calibration = model.calibrate(
    x_train[:num_calibration_samples],
    y_train[:num_calibration_samples],
    extra_metrics=["precision", "recall", "binary_accuracy"],
    verbose=1,
)